## Adapt Risks to Uses
Adapt the risks and mitigations matched from model cards to the use description.

In [2]:
from collections import defaultdict
import json
import numpy as np
import pandas as pd
import os
from copy import deepcopy
import time
import ast
import re
import requests
from openai import OpenAI

### OpenAI setup

In [2]:
from openai import OpenAI

# Get the first key from the uploaded dictionary
env_file_key = "openai.env"

# Open the file and read its content
with open(env_file_key, 'r', encoding='utf-8') as file:
    env_content = file.read()

api_key = env_content

client = OpenAI(
    # # This is the default and can be omitted
    api_key=api_key,
)


In [3]:
def chat_gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=prompt
    )
    return response.choices[0].message.content.strip()

### Prompt

In [4]:

# MESSAGES = [ { 'role': 'system', 'content': """
# Consider the following definitions: 
#     1) An AI incident is an event, circumstance or series of events where the development, use or malfunction of one or more AI systems directly or indirectly leads to any of the following harms: (a) injury or harm to the health of a person or groups of people; (b) disruption of the management and operation of critical infrastructure; (c) violations to human rights or a breach of obligations under the applicable law intended to protect fundamental, labour and intellectual property rights; (d) harm to property, communities or the environment.' The harm can be physical, psychological, reputational, economic/financial (including harm to property), environmental, public interest (e.g., protection of critical infrastructure and democratic institutions), human rights and fundamental rights. 
#     2) An AI risk is expressed as likelihood that harm or damage will occur. Risk is a function of both the probability of an event occurring and the severity of the consequences that would result. Risk is usually expressed in terms of risk sources, potential events, their consequences and their likelihood.  """ },

# { 'role': 'user', 'content': """You are provided in input with cardID, model description, a potential use of the model and a list of risks :
# cardID: "{}", model_description: "{}", model_task: "{}", use: "{}", ai_user: "{}", ai_subject: "{}", institution: "{}" model_risks: "{}", aiid_risks: "{}".

# Tasks:

# (1) The main purpose of this task is to rephrase each unique risk to adapt it to the use provided. Output the risk ONLY AFTER ADAPTING IT TO THE USE. If some risks are redundant and similar to each other, skip the duplicates and rephrase ONLY UNIQUE risks. If some risks do not pertain to the model description or use, skip those. DO NOT INVENT ANY NEW RISKS. 
#  Guidelines:
#  1. Understand Model Description, Model Task and Use: Thoroughly read and understand the specific context of the AI model being described, the task it is intended for and its application described in use.
#  2. Identify Relevant Risks: There are two lists of risks. Identify unique risks from both lists that are relevant to the described model and use; skip those that are not. Remember the source of the risk. For the model_risks, check if it is relevant to the model description and model_task. If it is not, skip those. For the aiid_risks, check if it is relevant to the model_task and use. Skip those, if it is not relevant.
#  3. Adapt Each Risk: Adapt each identified risk to the context of the model and use, ensuring it aligns with the specifics of the use. If the risk includes specific info which cannot be found in the model/use description, then do not impose the specificity on the adapted risk. Pay attention to the model_task. If the risk could not emanate from this type of model, do not adapt this risk.
#  4. Format and Clarity: Verb + Object + [Explanation]. Start with an action verb in active present tense (e.g., undermines, discriminates, infringes, reduces, increases, underperforms, etc.). Be clear, and to-the-point, with a maximum of 20 words.
#  5. Identify the subject at risk: For each adapted risk, identify the subject at risk from the list provided in input - ai_user, ai_subject and institution.
 
#  Examples of Good Adaptation: 
#  1. The use description involves a facial recognition system.
#     Reference Risk: Violates privacy rights.
#     Adapted Risk: Undermines the right to privacy if the facial recognition data is not properly secured.
#     Reasoning: Specifies how the privacy violation pertains to the facial recognition data. 
#  2. The model description involves a model fine-tuned on Western zodiac signs,
#     Reference Risk: Overfits heavily due to training on a very small corpus.
#     Adapted Risk: Overfits heavily due to fine-tuning on a small corpus of Western zodiac signs data.
#     Reasoning: Specifies the small corpus context of Western zodiac signs.

#  Examples of Bad Adaptation:
#  1. The model description involves a model fine-tuned on Twitter dataset
#     Reference Risk: Produces harmful content such as conspiracist views.
#     Bad Adaptation: Generates harmful conspiratorial content.
#     Good Adaptation: Produces harmful content such as conspiracist views common in Twitter data.
#     Reasoning: The good adaptation specifies the source of the harmful content, aligning it with the Twitter data context.
#  2. The use description involves a educational use case and model task is image classification
#     Reference Risk: Underperforms due to limited genuine samples for Arabic.
#     Bad Adaptation: Underperforms with educational content in Arabic due to limited data.
#     Reasoning: This risk needs to be skipped as the model does not have anything to do with Arabic.
#  3. The model task is image classification and use description involves a educational use case.
#     Reference Risk: Generates offensive content harming reputations.
#     Bad Adaptation: Generates offensive educational content impacting reputations.
#     Reasoning: This risk needs to be skipped as the model is not about image generation.
 
# Output Format: Ensure your output strictly follows this JSON structure.

# {{
#     "cardID": "<Card ID>",
#     "use": "<use>"
#     "risks": 
#     {{
#         [
#             {{
#                 "reference_risk": "Reference risk 1 from list",
#                 "source": "model_risks or aiid_risks",
#                 "adapted_risk": "Unique risk 1 adapted",
#                 "reasoning": "reasoning",
#                 "subject_at_risk": "<ai_user> or <ai_subject> or <institution>"
#             }},
#             {{
#                 "reference_risk": "Reference risk 2 from list",
#                 "source": "model_risks or aiid_risks",
#                 "adapted_risk": "Unique risk 2 adapted",
#                 "reasoning": "reasoning",
#                 "subject_at_risk": "<ai_user> or <ai_subject> or <institution>"
#             }},
#             ...
#         ]
#     }},
# }}


# Important Notes: Do not report your reasoning steps or any preamble like 'Here is the output', '```json ...```', ONLY the JSON result. In scenarios where there are no sentences mentioned, provide an empty JSON array for those sections.

# *** Double Check your output that it contains only the requested JSON and nothing else. *** """

# } ]

In [4]:

MESSAGES = [ { 'role': 'system', 'content': """
Consider the following definitions: 
    1) An AI incident is an event, circumstance or series of events where the development, use or malfunction of one or more AI systems directly or indirectly leads to any of the following harms: (a) injury or harm to the health of a person or groups of people; (b) disruption of the management and operation of critical infrastructure; (c) violations to human rights or a breach of obligations under the applicable law intended to protect fundamental, labour and intellectual property rights; (d) harm to property, communities or the environment.' The harm can be physical, psychological, reputational, economic/financial (including harm to property), environmental, public interest (e.g., protection of critical infrastructure and democratic institutions), human rights and fundamental rights. 
    2) An AI risk is expressed as likelihood that harm or damage will occur. Risk is a function of both the probability of an event occurring and the severity of the consequences that would result. Risk is usually expressed in terms of risk sources, potential events, their consequences and their likelihood.  """ },

{ 'role': 'user', 'content': """You are provided in input with cardID, model description, a potential use of the model and a list of risks :
cardID: "{}", model_description: "{}", model_task: "{}", use: "{}", ai_user: "{}", ai_subject: "{}", institution: "{}" model_risks: "{}", aiid_risks: "{}".

Tasks:

(1) The main purpose of this task is to map each unique risk to the use provided. Output the risk ONLY AFTER MAPPING IT TO THE USE. If some risks are redundant and similar to each other, skip the duplicates and map ONLY UNIQUE risks. If some risks do not pertain to the model description or use, skip those. DO NOT INVENT ANY NEW RISKS. 
 Guidelines:
 1. Understand Model Description, Model Task and Use: Thoroughly read and understand the specific context of the AI model being described, the task it is intended for and its application described in use.
 2. Identify Relevant Risks: There are two lists of risks. Identify unique risks from both lists that are relevant to the described model and use; skip those that are not. Remember the source of the risk. For the model_risks, check if it is relevant to the model description and model_task. If it is not, skip those. For the aiid_risks, check if it is relevant to the model_task and use. Skip those, if it is not relevant.
 3. Identify the subject at risk: For each mapped risk, identify the subject at risk from the list provided in input - ai_user, ai_subject and institution.
 
 Examples: 
 1. The use description involves a facial recognition system.
    Reference Risk: Violates privacy rights.
    Maps to the use: Yes
    Reasoning: Privacy violation pertains to the facial recognition data. 
 2. The model description involves a model fine-tuned on Western zodiac signs,
    Reference Risk: Overfits heavily due to training on a very small corpus.
    Maps to the use: Yes
    Reasoning: Pertains to the small corpus context of Western zodiac signs.
 3. The use description involves a educational use case and model task is image classification
    Reference Risk: Underperforms due to limited genuine samples for Arabic.
    Maps to the use: No
    Reasoning: This risk needs to be skipped as the model does not have anything to do with Arabic.
 3. The model task is image classification and use description involves a educational use case.
    Reference Risk: Generates offensive content harming reputations.
    Maps to the use: No
    Reasoning: This risk needs to be skipped as the model task is not about image generation.
 
Output Format: Ensure your output strictly follows this JSON structure. Ouput only relevant risks. No need to output skipped risks.

{{
    "cardID": "<Card ID>",
    "use": "<use>"
    "risks": 
    {{
        [
            {{
                "reference_risk": "Reference risk 1 from list",
                "source": "model_risks or aiid_risks",
                "reasoning": "reasoning",
                "subject_at_risk": "<ai_user> or <ai_subject> or <institution>"
            }},
            {{
                "reference_risk": "Reference risk 2 from list",
                "source": "model_risks or aiid_risks",
                "reasoning": "reasoning",
                "subject_at_risk": "<ai_user> or <ai_subject> or <institution>"
            }},
            ...
        ]
    }},
}}


Important Notes: Do not report your reasoning steps or any preamble like 'Here is the output', '```json ...```', ONLY the JSON result. In scenarios where there are no sentences mentioned, provide an empty JSON array for those sections.

*** Double Check your output that it contains only the requested JSON and nothing else. *** """

} ]

In [5]:
def format_prompt(MESSAGES, cardID, desc, task, use, ai_user, ai_subject, institution, model_risks, aiid_risks): 
    messages = deepcopy(MESSAGES) 
    messages[1]['content'] = messages[1]['content'].format(cardID, desc, task, use, ai_user, ai_subject, institution, model_risks, aiid_risks) 
    return messages

### Prepare data

In [30]:
# data_file = "data/new_data/results/matched_risks_besttest_aiid_Linq-Embed-Mistral_top5.csv"
# data_file = "data/new_data/results/matched_risks_user_study_aiid_Linq-Embed-Mistral_top10_12percent_taxonomy.csv"
data_file = "data/new_data/results/matched_risks_user_study_aiid_Linq-Embed-Mistral_top10_taxonomy.csv"
card_file = "data/new_data/unique_risks_without_model_cards_v3.json"
data = pd.read_csv(data_file, index_col="test_index")
risk_cards = pd.read_json(card_file)

In [31]:
data.drop('Unnamed: 0', axis=1, inplace=True )
data.dropna(subset="use1", inplace=True, axis=0)

In [32]:
risk_cards.head(2)

,modelId,author,creation_time,last_modified,downloads,likes,library_name,tags,pipeline_tag,card_data_language,card_data_license,card_data_library_name,card_data_tags,card_data_base_model,card_data_datasets,model_card_metadata,risks_limitations_bias,risk_section_len,model_description,model_desc_len
2,facebook/fasttext-language-identification,facebook,2023-03-06 12:52:50,2023-06-09T12:39:43+00:00,53352708,135,fasttext,"[fasttext, text-classification, language-ident...",text-classification,None,cc-by-nc-4.0,fasttext,"[text-classification, language-identification]",None,None,"{'base_model': None, 'datasets': None, 'eval_r...",### Limitations and bias\n\nEven if the traini...,786,## Model description\n\nfastText is a library...,741
4,google-bert/bert-base-uncased,google-bert,2022-03-02 23:29:04,2024-02-19T11:06:12+00:00,44931654,1678,transformers,"[transformers, pytorch, tf, jax, rust, coreml,...",fill-mask,en,apache-2.0,None,[exbert],None,"[bookcorpus, wikipedia]","{'base_model': None, 'datasets': ['bookcorpus'...",### Limitations and bias\n\nEven if the traini...,1783,## Model description\n\nBERT is a transformer...,1498


In [33]:
data

,test_description,similar_indices,matched_description,matched_risks,matched_axis1_target_of_analysis,matched_axis2_risk_area_main,matched_axis3_module_main,matched_mitigations,matched_risk_sections,test_risks,test_mitigations,similar_indices_aiid,matched_description_aiid,matched_risks_aiid,use1,use2,use3,use4
test_index,,,,,,,,,,,,,,,,,,
44,## Model description\n\nOPT was predominantl...,"[528, 769, 308, 927, 1334, 3114, 3229, 3457, 9...",[' ## Model description\n\nOPT was predominan...,Underperforms in nonresearch environments** In...,capability** capability** capability** capabil...,Representation & Toxicity Harms** Representati...,Language Model Module** Language Model Module*...,** ** ** ** ** ** ** Perform responsible best ...,### Limitations and bias\n\nAs mentioned in Me...,Contains unfiltered content from the internet ...,NaN,"[399, 357, 339, 85, 532, 367, 259, 443, 473, 179]",['Meta AI trained and hosted a scientific pape...,Violates academic policies by using unauthoriz...,"{\n ""Use"": 11,\n ""Domain"": ""Reco...","{\n ""Use"": 16,\n ""Domain"": ""Mar...","{\n ""Use"": 8,\n ""Domain"": ""Educa...","{\n ""Use"": 14,\n ""Domain"": ""Arts..."
82,## Model Summary\n\nThe Phi-...,"[36138, 532, 56473, 1125, 58339, 43506, 29344,...",[' ## Model Summary\r\n\r\nThe...,Erases representation of some groups** Underpe...,capability** capability** systemic** capabilit...,Representation & Toxicity Harms** Representati...,Language Model Module** Language Model Module*...,Implement additional mitigations for sensitive...,## Responsible AI Considerations\r\n\r\nLike o...,Underperforms on non-English languages ** Unde...,Implement additional mitigations for sensitive...,"[262, 473, 356, 552, 468, 571, 287, 278, 85, 399]",['Publicly deployed open-source model DALL-E M...,Undermines security measures by solving CAPTCH...,"{\n ""Use"": 11,\n ""Domain"": ""Reco...","{\n ""Use"": 16,\n ""Domain"": ""...","{\n ""Use"": 8,\n ""Domain"": ""Educa...","{\n ""Use"": 12,\n ""Domain"": ""Soci..."
93,## Model Description\n\n`Stable Beluga 2` is...,"[9679, 6187, 5947, 6152, 942, 26314, 5816, 325...",[' ## Model Description\n\n`Stable Beluga 1` i...,Generates nondeterministic responses due to mo...,capability** capability** capability** capabil...,Misinformation Harms** Human Autonomy & Integr...,Language Model Module** Language Model Module*...,Perform safety testing ** Tune model to specif...,### Ethical Considerations and Limitations\n\n...,Carries risks with use ** Underperforms in non...,Perform safety testing ** Perform tuning tailo...,"[314, 578, 451, 421, 278, 465, 505, 179, 399, ...","['Stable Diffusion, an open-source image gener...",Violates copyright law by unauthorized use of ...,"{\n ""Use"": 11,\n ""Domain"": ""Reco...","{\n ""Use"": 16,\n ""Domain"": ""...","{\n ""Use"": 8,\n ""Domain"": ""Educa...","{\n ""Use"": 4,\n ""Domain"": ""H..."
231,## Model description\n\nThe text-to-video gen...,"[225146, 40258, 47919, 23293, 26746, 34506, 23...",[' ## Model Details\n\n* **Developed by:** [L...,Cannot render legible text** Biases towards wh...,capability** capability** capability** capabil...,Information & Safety Harms** Representation & ...,Output Module** Language Model Module** Langua...,** ** Approach DeciDiffusion with discretion. ...,## Limitations\n\n* Limited knowledge of tempo...,Underperforms in generating realistic represen...,Prohibit generating demeaning or harmful conte...,"[314, 486, 421, 529, 465, 504, 451, 578, 179, ...","['Stable Diffusion, an open-source image gener...",Violates copyright law by unauthorized use of ...,"{\n ""Use"": 12,\n ""Domain"": ""Soci...","{\n ""Use"": 16,\n ""Domain"": ""Mark...","{\n ""Use"": 8,\n ""Domain"": ""Educa...","{\n ""Use"": 5,\n ""Domain"": ""Well-..."
1490,# Model Description\n- **Developed by**: Natur...,"[12415, 13198, 259, 54, 4480, 6407, 129265, 76...",[' # Model Description\n- **Developed by**: Na...,Cannot render legible text** Underperforms wit...,capability** capability** capabi

In [40]:
def remove_newlines_and_indentations(input_string):
    # Replace newline characters with an empty string
    no_newlines = input_string.replace("\n", "")
    
    # Strip leading and trailing spaces from each line and concatenate them
    no_indentations = "".join(line.strip() for line in no_newlines.splitlines())
    
    return no_indentations

In [41]:
ast.literal_eval(remove_newlines_and_indentations(data["use2"][82]))

{'Use': 16,
 'Domain': 'Marketing and Advertising',
 'Purpose': 'Creating personalized ad campaigns',
 'Capability': 'Analyzing user behavior from social media posts',
 'AI User': 'Marketing agencies',
 'AI Subject': 'Consumers'}

In [62]:
use = "**Urban Planning** **Purpose:** Designing pedestrian-friendly areas **Capability:** Identifying sidewalk usage from images **AI User:** City planners **AI Subject:** City residents"
data["test_description"][42785], data["matched_risks"][42785], data["matched_risks_aiid"][42785], risk_cards["pipeline_tag"][42785]

('## Model Details\n\n+ **Model Name**: Segments-Sidewalk-SegFormer-B0\n+ **Model Type**: Semantic Segmentation\n+ **Base Model**: nvidia/segformer-b0-finetuned-ade-512-512\n+ **Fine-Tuning Dataset**: Sidewalk-Semantic ## Model Description\n\nThe **Segments-Sidewalk-SegFormer-B0** model is a semantic segmentation model fine-tuned on the **sidewalk-semantic** dataset. It is based on the **SegFormer (b0-sized)** architecture and has been adapted for the task of segmenting sidewalk images into various classes, such as road surfaces, pedestrians, vehicles, and more.      ',
 'Reinforces social biases $$ Exacerbates social biases $$ Lacks diversity in dataset $$ Is not representative in dataset $$ Focuses on celebrities only $$ ** ** ** ** Underperforms on images with different characteristics from the training dataset. $$ Struggles with very small or very large objects in images. $$ Inherits biases present in the training data, such as underrepresentation of certain classes. $$ Leads to ha

In [55]:
use = "**Education and vocational training** **Purpose:** Providing visual learning aids **Capability:** Classifying educational images **AI User:** Teachers, educational platforms **AI Subject:** Students"
data["test_description"][5], data["matched_risks"][5], data["matched_risks_aiid"][5]

(' ## Model Details\n\nThe CLIP model was developed by researchers at OpenAI to learn about what contributes to robustness in computer vision tasks. The model was also developed to test the ability of models to generalize to arbitrary image classification tasks in a zero-shot manner. It was not developed for general model deployment - to deploy models like CLIP, researchers will first need to carefully study their capabilities in relation to the specific context they’re being deployed within.                  ',
 'Undermines safety when deployed without thorough in-domain testing. $$ Undermines fair use due to lack of testing norms and checks. $$ Underperforms on non-English languages. $$ Underperforms on fine-grained classification and counting objects. $$ Poses issues with fairness and bias. $$ Underestimates model performance due to use of linear probes. $$ Exhibits biases depending on class design and category choices. $$ Discriminates based on race and gender in denigration tasks.

In [67]:
use = "**Essential private services and public services and benefits** **Purpose:** Improving public infrastructure **Capability:** Assessing sidewalk accessibility **AI User:** Municipal authorities **AI Subject:** Citizens"
data["test_description"][42785], data["matched_risks"][42785], data["matched_risks_aiid"][42785], risk_cards["pipeline_tag"][42785]

('## Model Details\n\n+ **Model Name**: Segments-Sidewalk-SegFormer-B0\n+ **Model Type**: Semantic Segmentation\n+ **Base Model**: nvidia/segformer-b0-finetuned-ade-512-512\n+ **Fine-Tuning Dataset**: Sidewalk-Semantic ## Model Description\n\nThe **Segments-Sidewalk-SegFormer-B0** model is a semantic segmentation model fine-tuned on the **sidewalk-semantic** dataset. It is based on the **SegFormer (b0-sized)** architecture and has been adapted for the task of segmenting sidewalk images into various classes, such as road surfaces, pedestrians, vehicles, and more.      ',
 'Reinforces social biases $$ Exacerbates social biases $$ Lacks diversity in dataset $$ Is not representative in dataset $$ Focuses on celebrities only $$ ** ** ** ** Underperforms on images with different characteristics from the training dataset. $$ Struggles with very small or very large objects in images. $$ Inherits biases present in the training data, such as underrepresentation of certain classes. $$ Leads to ha

In [68]:
messages = format_prompt(MESSAGES, cardID=42785, desc=data["test_description"][42785], task=risk_cards["pipeline_tag"][42785], use=use, model_risks=data["matched_risks"][42785], aiid_risks=data["matched_risks_aiid"][42785])

In [69]:
messages

[{'role': 'system',
  'content': "\nConsider the following definitions: \n    1) An AI incident is an event, circumstance or series of events where the development, use or malfunction of one or more AI systems directly or indirectly leads to any of the following harms: (a) injury or harm to the health of a person or groups of people; (b) disruption of the management and operation of critical infrastructure; (c) violations to human rights or a breach of obligations under the applicable law intended to protect fundamental, labour and intellectual property rights; (d) harm to property, communities or the environment.' The harm can be physical, psychological, reputational, economic/financial (including harm to property), environmental, public interest (e.g., protection of critical infrastructure and democratic institutions), human rights and fundamental rights. \n    2) An AI risk is expressed as likelihood that harm or damage will occur. Risk is a function of both the probability of an ev

In [70]:
response = chat_gpt(messages)

In [71]:
response

'{\n    "cardID": "42785",\n    "use": "**Essential private services and public services and benefits** **Purpose:** Improving public infrastructure **Capability:** Assessing sidewalk accessibility **AI User:** Municipal authorities **AI Subject:** Citizens",\n    "risks": \n    [\n        {\n            "reference_risk": "Reinforces social biases",\n            "source": "model_risks",\n            "adapted_risk": "Reinforces social biases in assessing sidewalk accessibility.",\n            "reasoning": "The model may inherit biases from the training data, affecting the fairness of sidewalk accessibility assessments.",\n            "subject_at_risk": "AI subject"\n        },\n        {\n            "reference_risk": "Lacks diversity in dataset",\n            "source": "model_risks",\n            "adapted_risk": "Lacks diversity in dataset, undermining sidewalk accessibility evaluations.",\n            "reasoning": "A non-representative dataset may not account for diverse sidewalk cond

### Batch the data

In [47]:
def batch_data(data=data):

    tasks = []
    for i, row in data.iterrows():
        print (i)
        for u in ["use1", "use2", "use3", "use4"]:
            use = ast.literal_eval(remove_newlines_and_indentations(row[u]))
            ai_user = use['AI User']
            ai_subj = use['AI Subject']
            use = "Domain-- "+use['Domain']+', '+"Purpose-- "+use['Purpose']+', '+"Capability-- "+use['Capability']
            print (use) 
            messages = format_prompt(MESSAGES, cardID=i, desc=row["test_description"], task=risk_cards["pipeline_tag"][i], use=use, ai_user=ai_user, ai_subject=ai_subj, institution="Institution and Environment", model_risks=row["matched_risks"], aiid_risks=row["matched_risks_aiid"])

            task = {
            "custom_id": f"task-{int(i)}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                # This is what you would have in your Chat Completions API call
                "model": "gpt-4o",
                "temperature": 0,
                "response_format": { 
                    "type": "json_object"
                },
                "messages": messages,
                }
            }
    
            tasks.append(task)

    return tasks

In [48]:
# Creating the file

file_name = "batch_risks.jsonl"

tasks = batch_data(data=data)
with open(file_name, 'w') as file:
    for obj in tasks:
        file.write(json.dumps(obj) + '\n')

44
Domain-- Recommender Systems and Personalization, Purpose-- Recommending personalized content, Capability-- Analyzing user preferences and suggesting items
Domain-- Marketing and Advertising, Purpose-- Creating personalized ad copy, Capability-- Generating tailored messages from user data
Domain-- Education and vocational training, Purpose-- Personalizing learning experiences, Capability-- Analyzing student performance and tailoring content
Domain-- Arts and Entertainment, Purpose-- Generating creative writing prompts, Capability-- Analyzing themes and creating prompts
82
Domain-- Recommender Systems and Personalization, Purpose-- Recommending personalized content, Capability-- Analyzing preferences for suggestions
Domain-- Marketing and Advertising, Purpose-- Creating personalized ad campaigns, Capability-- Analyzing user behavior from social media posts
Domain-- Education and vocational training, Purpose-- Personalizing learning experiences, Capability-- Analyzing student performa

In [49]:
len(tasks)

20

### Upload batch file

In [108]:
batch_file = client.files.create(
  file=open(file_name, "rb"),
  purpose="batch"
)

In [109]:
batch_file

FileObject(id='file-uwVuhNk1r6umjV86gmQ8aSmb', bytes=91356, created_at=1722117795, filename='batch_risks.jsonl', object='file', purpose='batch', status='processed', status_details=None)

In [50]:
responses = []
for task in tasks:
    messages = task["body"]["messages"]
    response = chat_gpt(messages)
    responses.append(response)

In [52]:
responses

['{\n    "cardID": "44",\n    "use": "Domain-- Recommender Systems and Personalization, Purpose-- Recommending personalized content, Capability-- Analyzing user preferences and suggesting items",\n    "risks": [\n        {\n            "reference_risk": "Induces bias and safety issues due to lack of diverse training data",\n            "source": "model_risks",\n            "reasoning": "Lack of diverse training data can result in biased recommendations which can influence user preferences unfairly.",\n            "subject_at_risk": "ai_subject"\n        },\n        {\n            "reference_risk": "Produces low diversity and hallucinated outputs",\n            "source": "model_risks",\n            "reasoning": "Low diversity in generated content can limit the range of personalized recommendations, potentially reducing user engagement.",\n            "subject_at_risk": "ai_subject"\n        },\n        {\n            "reference_risk": "Has quality issues in terms of generation diversity

In [53]:
file_name = "data/new_data/results/user_study_risks.jsonl"

with open(file_name, 'w') as file:
    for obj in responses:
        file.write(json.dumps(obj) + '\n')

### Create batch job

In [110]:
# Creating the batch job
batch_job = client.batches.create(
  input_file_id=batch_file.id,
  endpoint="/v1/chat/completions",
  completion_window="24h"
)

In [113]:
batch_job

Batch(id='batch_ST4JywDdQmjUaBnDCQcfcuQy', completion_window='24h', created_at=1722117813, endpoint='/v1/chat/completions', input_file_id='file-uwVuhNk1r6umjV86gmQ8aSmb', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1722204213, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [81]:
batch_job = client.batches.retrieve('batch_Tc3c7WfqCSsJDVs3hjK5e8DI')

In [82]:

# Checking batch status
batch_job = client.batches.retrieve(batch_job.id)
print(batch_job)

Batch(id='batch_Tc3c7WfqCSsJDVs3hjK5e8DI', completion_window='24h', created_at=1722110349, endpoint='/v1/chat/completions', input_file_id='file-uhLN61DZof0aTQlj0MopgXO8', object='batch', status='failed', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=Errors(data=[BatchError(code='duplicate_custom_id', line=2, message='The custom_id for this request is a duplicate of another request. The custom_id parameter must be unique for each request in a batch.', param='custom_id'), BatchError(code='duplicate_custom_id', line=3, message='The custom_id for this request is a duplicate of another request. The custom_id parameter must be unique for each request in a batch.', param='custom_id'), BatchError(code='duplicate_custom_id', line=5, message='The custom_id for this request is a duplicate of another request. The custom_id parameter must be unique for each request in a batch.', param='custom_id'), BatchError(code='duplicate_custom_id', line=6, message='The cu

### Retrieve results

In [159]:
result_file_id = batch_job.output_file_id
result = client.files.content(result_file_id).content

In [1]:
results_file_name = "data/new_data/results/user_study_risks.jsonl"
results_file = "user_study/final_study/user_study_risks_v2.json"


In [55]:
results_file_name, results_file

('data/new_data/results/user_study_risks.jsonl',
 'data/new_data/results/user_study_risks_v2.json')

In [56]:
# Loading data from saved file
results = []
with open(results_file_name, 'r') as file:
    for line in file:
        # Parsing the JSON string into a dict and appending to the list of results
        json_object = json.loads(line.strip())
        results.append(json_object)

In [57]:
for res in results:
    print (res)
    break

{
    "cardID": "44",
    "use": "Domain-- Recommender Systems and Personalization, Purpose-- Recommending personalized content, Capability-- Analyzing user preferences and suggesting items",
    "risks": [
        {
            "reference_risk": "Induces bias and safety issues due to lack of diverse training data",
            "source": "model_risks",
            "reasoning": "Lack of diverse training data can result in biased recommendations which can influence user preferences unfairly.",
            "subject_at_risk": "ai_subject"
        },
        {
            "reference_risk": "Produces low diversity and hallucinated outputs",
            "source": "model_risks",
            "reasoning": "Low diversity in generated content can limit the range of personalized recommendations, potentially reducing user engagement.",
            "subject_at_risk": "ai_subject"
        },
        {
            "reference_risk": "Has quality issues in terms of generation diversity",
            "sou

In [58]:
FULL_RES = []
cnt_errors = []

# Reading only the first results
for res in results:
    try:
        res = ast.literal_eval(res)
        # print (index)
        # res["Incident ID"] = int(index)
        FULL_RES.append(res)
    except Exception as e:
        print (e)
        # print (result)
        cnt_errors.append(res)
        continue

In [59]:
cnt_errors

[]

In [60]:
FULL_RES

[{'cardID': '44',
  'use': 'Domain-- Recommender Systems and Personalization, Purpose-- Recommending personalized content, Capability-- Analyzing user preferences and suggesting items',
  'risks': [{'reference_risk': 'Induces bias and safety issues due to lack of diverse training data',
    'source': 'model_risks',
    'reasoning': 'Lack of diverse training data can result in biased recommendations which can influence user preferences unfairly.',
    'subject_at_risk': 'ai_subject'},
   {'reference_risk': 'Produces low diversity and hallucinated outputs',
    'source': 'model_risks',
    'reasoning': 'Low diversity in generated content can limit the range of personalized recommendations, potentially reducing user engagement.',
    'subject_at_risk': 'ai_subject'},
   {'reference_risk': 'Has quality issues in terms of generation diversity',
    'source': 'model_risks',
    'reasoning': 'Quality issues in generation can impact the precision and relevance of recommended content.',
    'su

In [61]:
###############################
# save result
with open(results_file, "w") as json_file:
    json.dump(FULL_RES, json_file, indent=4)  # 4 spaces of indentation

### Sort risks according to uses

In [18]:
mapped_risks = pd.read_json(results_file)

# Normalize the JSON mapped_risks to create a flat DataFrame
df = pd.json_normalize(mapped_risks.explode('risks').to_dict(orient='records'))

# Define a function to extract the capability from the use
def extract_capability(use):
    # Extract the capability part from the structured use string
    capability_part = use.split('Capability-- ')[1].strip() if 'Capability-- ' in use else None
    return capability_part

# Group by 'cardID' and 'risks.reference_risk' and aggregate the uses
risk_counts = df.groupby(['cardID', 'risks.reference_risk', 'risks.source']).agg(
    count=('use', 'nunique'),  # Count the number of unique uses
    uses=('use', lambda x: list(x.unique())),  # List of unique uses
    use_capability=('use', lambda x: list(x.apply(extract_capability).unique()))  # List of unique capabilities
).reset_index()

# Sort the counts in descending order for each cardID
risk_counts_sorted = risk_counts.sort_values(by=['cardID', 'count'], ascending=[True, False])

# Save the result to a CSV file
output_risk_file = 'user_study/final_study/user_study_reference_risk_counts_sorted.csv'
risk_counts_sorted.to_csv(output_file, index=False)
# Display the result
risk_counts_sorted.head()


,cardID,risks.reference_risk,risks.source,count,uses,use_capability
24,44,Produces low diversity and hallucinated outputs,model_risks,4,[Domain-- Recommender Systems and Personalizat...,[Analyzing user preferences and suggesting ite...
0,44,Contains biased training data from the internet,model_risks,3,[Domain-- Recommender Systems and Personalizat...,[Analyzing user preferences and suggesting ite...
3,44,Exhibits biases,model_risks,3,[Domain-- Recommender Systems and Personalizat...,[Analyzing user preferences and suggesting ite...
6,44,Facilitates misinformation spreading false kno...,aiid_risks,3,[Domain-- Recommender Systems and Personalizat...,[Analyzing user preferences and suggesting ite...
8,44,Generates factual inaccuracies,model_risks,3,"[Domain-- Marketing and Advertising, Purpose--...","[Generating tailored messages from user data, ..."


### Mitigation mapping

In [62]:

MESSAGES = [ { 'role': 'system', 'content': """
Consider the following definitions: 
    1) An AI incident is an event, circumstance or series of events where the development, use or malfunction of one or more AI systems directly or indirectly leads to any of the following harms: (a) injury or harm to the health of a person or groups of people; (b) disruption of the management and operation of critical infrastructure; (c) violations to human rights or a breach of obligations under the applicable law intended to protect fundamental, labour and intellectual property rights; (d) harm to property, communities or the environment.' The harm can be physical, psychological, reputational, economic/financial (including harm to property), environmental, public interest (e.g., protection of critical infrastructure and democratic institutions), human rights and fundamental rights. 
    2) An AI risk is expressed as likelihood that harm or damage will occur. Risk is a function of both the probability of an event occurring and the severity of the consequences that would result. Risk is usually expressed in terms of risk sources, potential events, their consequences and their likelihood.  """ },

{ 'role': 'user', 'content': """You are provided in input with cardID, model description, a potential use of the model and a list of risks :
cardID: "{}", model_description: "{}", model_task: "{}", model_risks: "{}", aiid_risks: "{}", mitigations: "{}".

Tasks:

(1) The main purpose of this task is to map each unique risk to the mitigations provided. Output the risk ONLY AFTER MAPPING IT. If some risks are redundant and similar to each other, skip the duplicates and map ONLY UNIQUE risks. If some risks do not pertain to the model description, skip those. DO NOT INVENT ANY NEW RISKS. 
 Guidelines:
 1. Understand Model Description, Model Task: Thoroughly read and understand the specific context of the AI model being described, the task it is intended for.
 2. Identify Unique Risks: There are two lists of risks. Identify unique risks from both lists that are relevant to the described model; skip those that are not. Remember the source of the risk. For the model_risks, check if it is relevant to the model description and model_task. If it is not, skip those. For the aiid_risks, check if it is relevant to the model_task. Skip those, if it is not relevant.
 3. Map Relevant Risks to Mitigations: For each of the identified risk, map the appropriate mitigation strategy given. Note that each mitigation can map to more than one risk.
 
 Examples: 
 1. Reference Risk: Violates privacy rights.
    Mitigation mapped: Limit data collection to essential information only.
 
 2. Reference Risk: Generates offensive content harming reputations.
    Mitigation mapped: Probe these aspects on your use-cases to evaluate risks.
 
Output Format: Ensure your output strictly follows this JSON structure. Ouput only relevant risks. No need to output skipped risks.

{{
    "cardID": "<Card ID>",
    "use": "<use>"
    "risks": 
    {{
        [
            {{
                "reference_risk": "Reference risk 1 from list",
                "source": "model_risks or aiid_risks",
                "reasoning": "reasoning",
                "subject_at_risk": "<ai_user> or <ai_subject> or <institution>"
            }},
            {{
                "reference_risk": "Reference risk 2 from list",
                "source": "model_risks or aiid_risks",
                "reasoning": "reasoning",
                "subject_at_risk": "<ai_user> or <ai_subject> or <institution>"
            }},
            ...
        ]
    }},
}}


Important Notes: Do not report your reasoning steps or any preamble like 'Here is the output', '```json ...```', ONLY the JSON result. In scenarios where there are no sentences mentioned, provide an empty JSON array for those sections.

*** Double Check your output that it contains only the requested JSON and nothing else. *** """

} ]

In [ ]:
def format_prompt(MESSAGES, cardID, desc, task, model_risks, aiid_risks, mitigations): 
    messages = deepcopy(MESSAGES) 
    messages[1]['content'] = messages[1]['content'].format(cardID, desc, task, model_risks, aiid_risks, mitigations) 
    return messages

In [ ]:
def batch_data(data=data):
    tasks = []
    for i, row in data.iterrows():
        print (i)
    
        messages = format_prompt(MESSAGES, cardID=i, desc=row["test_description"], task=risk_cards["pipeline_tag"][i], model_risks=row["matched_risks"], aiid_risks=row["matched_risks_aiid"], mitigations=row["matched_mitigations"])
        task = {
        "custom_id": f"task-{int(i)}",
        "method": "POST",
        "url": "/v1/chat/completions",
        "body": {
            # This is what you would have in your Chat Completions API call
            "model": "gpt-4o",
            "temperature": 0,
            "response_format": { 
                "type": "json_object"
            },
            "messages": messages,
            }
        }
    
        tasks.append(task)
    return tasks


In [ ]:
file_name = "data/new_data/results/user_study_risks.jsonl"

with open(file_name, 'w') as file:
    for obj in responses:
        file.write(json.dumps(obj) + '\n')

In [2]:
results_mit_file_name = "data/new_data/results/user_study_mitigations.jsonl"
results_mit_file = "user_study/final_study/user_study_mitigations_v2.json"


In [3]:
results_mit_file_name, results_mit_file

('data/new_data/results/user_study_mitigations.jsonl',
 'user_study/final_study/user_study_mitigations_v2.json')

In [ ]:
# Loading data from saved file
results = []
with open(results_mit_file_name, 'r') as file:
    for line in file:
        # Parsing the JSON string into a dict and appending to the list of results
        json_object = json.loads(line.strip())
        results.append(json_object)

In [ ]:
for res in results:
    print (res)
    break

In [ ]:
FULL_RES = []
cnt_errors = []

# Reading only the first results
for res in results:
    try:
        res = ast.literal_eval(res)
        # print (index)
        # res["Incident ID"] = int(index)
        FULL_RES.append(res)
    except Exception as e:
        print (e)
        # print (result)
        cnt_errors.append(res)
        continue

In [ ]:
cnt_errors

[]

In [ ]:
FULL_RES

In [ ]:
###############################
# save result
with open(results_mit_file, "w") as json_file:
    json.dump(FULL_RES, json_file, indent=4)  # 4 spaces of indentation

In [19]:
filtered_mit_file = "user_study/final_study/user_study_filtered_mitigations_v2.json"
output_risk_file = "user_study/final_study/user_study_reference_risk_counts_sorted.csv"

# Load the original sorted risk and mitigations JSON files
df = pd.read_csv(output_risk_file)

with open(results_mit_file, 'r') as file:
    mitigations_data = json.load(file)

# Extract risks in the order they appear
ordered_reference_risks = df['risks.reference_risk'].tolist()

# Normalize JSON data into a DataFrame
mitigations_df = pd.json_normalize(mitigations_data, 'risks', ['cardID'])

# Create a lookup DataFrame for mitigations based on the reference risk
mitigations_lookup_df = mitigations_df[['reference_risk', 'cardID', 'risks.source', 'count', 'use_capability']]

# Filter the mitigations DataFrame based on the ordered reference risks
filtered_mitigations = pd.merge(
    pd.DataFrame({'reference_risk': ordered_reference_risks}),
    mitigations_lookup_df,
    on='reference_risk',
    how='left'
)

# Group by cardID and aggregate risks into lists, maintaining order
filtered_mitigations = filtered_mitigations.groupby('cardID').apply(
    lambda x: x[['reference_risk', 'source', 'count', 'uses']].to_dict(orient='records')
).reset_index(name='risks')

# Convert the DataFrame to the desired JSON format
filtered_mitigations_json = filtered_mitigations.to_dict(orient='records')

# # Filter the mitigations data based on the reference risks
# filtered_mitigations = []
# for card in mitigations_data:
#     filtered_risks = []
#     for risk in card['risks']:
#         if risk['reference_risk'] in reference_risks:
#             filtered_risks.append(risk)
#     if filtered_risks:
#         filtered_mitigations.append({
#             'cardID': card['cardID'],
#             'risks': filtered_risks
#         })

# Save the filtered mitigations data to a new JSON file
with open(filtered_mit_file, 'w') as outfile:
    json.dump(filtered_mitigations, outfile, indent=4)


KeyError: "['risks.source', 'count', 'use_capability'] not in index"

In [22]:
ordered_risks = {}
for _, row in risk_counts_sorted.iterrows():
    if row['cardID'] not in ordered_risks:
        ordered_risks[row['cardID']] = []
    ordered_risks[row['cardID']].append(row['risks.reference_risk'])

In [23]:
filtered_mit_file = "user_study/final_study/user_study_filtered_mitigations_v2_.json"
# output_risk_file = "user_study/final_study/user_study_reference_risk_counts_sorted.csv"

with open(results_mit_file, 'r') as file:
    mitigations_data = json.load(file)


# Extract the ordered reference_risks by cardID
ordered_risks = {}
for _, row in risk_counts_sorted.iterrows():
    if row['cardID'] not in ordered_risks:
        ordered_risks[row['cardID']] = []
    ordered_risks[row['cardID']].append(row['risks.reference_risk'])

# Filter and reorder the mitigations data based on the reference risks
filtered_mitigations = []
for card in mitigations_data:
    card_id = card['cardID']
    if card_id in ordered_risks:
        filtered_risks = []
        # Reorder the risks according to the ordered_risks list
        for reference_risk in ordered_risks[card_id]:
            for risk in card['risks']:
                if risk['reference_risk'] == reference_risk:
                    filtered_risks.append(risk)
        if filtered_risks:
            filtered_mitigations.append({
                'cardID': card['cardID'],
                'risks': filtered_risks
            })

# Save the filtered and ordered mitigations data to a new JSON file
with open(filtered_mit_file, 'w') as outfile:
    json.dump(filtered_mitigations, outfile, indent=4)

### Map risks and mitigations and sort according to uses

In [5]:
results_file = "user_study/final_study/user_study_risks_v2.json"
results_mit_file = "user_study/final_study/user_study_mitigations_v2.json"
output_risk_file = "user_study/final_study/user_study_reference_risk_counts_sorted_.csv"

# Load the risks and mitigations JSON files
with open(results_file, 'r') as file:
    risk_data = json.load(file)

with open(results_mit_file, 'r') as file:
    mitigations_data = json.load(file)

# Extract reference risks from the original JSON file
reference_risks = set()
for item in risk_data:
    for risk in item.get('risks', []):
        reference_risks.add(risk['reference_risk'])

# Filter the mitigations data based on the reference risks
filtered_mitigations = []
for card in mitigations_data:
    filtered_risks = []
    for risk in card['risks']:
        if risk['reference_risk'] in reference_risks:
            filtered_risks.append(risk)
    if filtered_risks:
        filtered_mitigations.append({
            'cardID': card['cardID'],
            'risks': filtered_risks
        })

# Save the filtered mitigations data to a temporary JSON file
temp_filtered_mitigations_file = 'user_study/final_study/temp_filtered_mitigations.json'
with open(temp_filtered_mitigations_file, 'w') as outfile:
    json.dump(filtered_mitigations, outfile, indent=4)

# Load the filtered mitigations into a DataFrame
mitigations_df = pd.read_json(temp_filtered_mitigations_file)

# Normalize the JSON mapped_risks to create a flat DataFrame
df_risks = pd.json_normalize(pd.DataFrame(risk_data).explode('risks').to_dict(orient='records'))
df_mitigations = pd.json_normalize(pd.DataFrame(filtered_mitigations).explode('risks').to_dict(orient='records'))

# Define a function to extract the capability from the use
def extract_capability(use):
    capability_part = use.split('Capability-- ')[1].strip() if 'Capability-- ' in use else None
    return capability_part

# Group by 'cardID' and 'risks.reference_risk' and aggregate the uses
risk_counts = df_risks.groupby(['cardID', 'risks.reference_risk', 'risks.source']).agg(
    count=('use', 'nunique'),  # Count the number of unique uses
    uses=('use', lambda x: list(x.unique())),  # List of unique uses
    use_capability=('use', lambda x: list(x.apply(extract_capability).unique()))  # List of unique capabilities
).reset_index()

# Merge the mitigations with the risk counts
merged_data = pd.merge(risk_counts, df_mitigations[['cardID', 'risks.reference_risk', 'risks.mapped_mitigations']],
                       on=['cardID', 'risks.reference_risk'], how='left')

# Sort the counts in descending order for each cardID
merged_data_sorted = merged_data.sort_values(by=['cardID', 'count'], ascending=[True, False])

# Save the result to a CSV file
merged_data_sorted.to_csv(output_risk_file, index=False)

# Display the result
merged_data_sorted.head()


,cardID,risks.reference_risk,risks.source,count,uses,use_capability,risks.mapped_mitigations
34,1490,Underperforms with non-English prompts,model_risks,3,"[Domain-- Social Media, Purpose-- Creating eng...","[Generating GIFs from user content, Generating...",NaN
0,1490,Cannot render legible text,model_risks,2,"[Domain-- Marketing and Advertising, Purpose--...","[Generating GIFs from marketing text, Generati...",NaN
1,1490,Contains adult material in training data,model_risks,2,"[Domain-- Social Media, Purpose-- Creating eng...","[Generating GIFs from user content, Generating...",[Promote safe and appropriate content generati...
3,1490,Creates hostile or alienating environments for...,model_risks,2,"[Domain-- Social Media, Purpose-- Creating eng...","[Generating GIFs from user content, Generating...",[Use Safety Checker to filter harmful concepts]
16,1490,Generates nonfactual or untrue representations...,model_risks,2,"[Domain-- Education and vocational training, P...","[Generating GIFs from lesson content, Generati...",NaN


In [4]:
merged_data_sorted

,cardID,risks.reference_risk,risks.source,count,uses,use_capability,risks.mapped_mitigations
34,1490,Underperforms with non-English prompts,model_risks,3,"[Domain-- Social Media, Purpose-- Creating eng...","[Generating GIFs from user content, Generating...",NaN
0,1490,Cannot render legible text,model_risks,2,"[Domain-- Marketing and Advertising, Purpose--...","[Generating GIFs from marketing text, Generati...",NaN
1,1490,Contains adult material in training data,model_risks,2,"[Domain-- Social Media, Purpose-- Creating eng...","[Generating GIFs from user content, Generating...",[Promote safe and appropriate content generati...
3,1490,Creates hostile or alienating environments for...,model_risks,2,"[Domain-- Social Media, Purpose-- Creating eng...","[Generating GIFs from user content, Generating...",[Use Safety Checker to filter harmful concepts]
16,1490,Generates nonfactual or untrue representations...,model_risks,2,"[Domain-- Education and vocational training, P...","[Generating GIFs from lesson content, Generati...",NaN
...,...,...,...,...,...,...,...
177,93,Small models may be more susceptible to halluc...,model_risks,1,"[Domain-- Health and Healthcare, Purpose-- Ass...",[Analyzing patient data and suggesting conditi...,NaN
178,93,Undermines trust in AI systems by mishandling ...,aiid_risks,1,[Domain-- Recommender Systems and Personalizat...,[Analyzing preferences for suggestions],NaN
179,93,Underperforms in non-English languages,model_risks,1,[Domain-- Recommender Systems and Personalizat...,[Analyzing preferences for suggestions],[Avoid using models unsuitable for your applic...
180,93,Used maliciously for generating disinformation...,model_risks,1,[Domain-- Recommender Systems and Personalizat...,[Analyzing preferences for suggestions],[Hope for better regulations and standards fro...
